In [0]:
import os
import numpy as np
import pandas as pd
import cv2
from zipfile import ZipFile 

file_name = '//9d34462453e311ea.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()  

In [4]:
 path = '/content/dataset'

 train_df = pd.read_csv(path+'/train.csv')
 test_df = pd.read_csv(path+'/test.csv')

 train_df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [5]:
class_map = {
    'Food': 0,
    'Attire': 1,
    'Decorationandsignage': 2,
    'misc': 3
}

inverse_class_map = {
    0: 'Food',
    1: 'Attire',
    2: 'Decorationandsignage',
    3: 'misc'
}

train_df['Class'] = train_df['Class'].map(class_map).astype(np.uint8)
train_df.head()

,Image,Class
0,image7042.jpg,0
1,image3327.jpg,3
2,image10335.jpg,1
3,image8019.jpg,0
4,image2128.jpg,1


In [0]:
train_path = '/content/dataset/Train Images/'
test_path = '/content/dataset/Test Images/'

h, w = 224, 224

train_images, train_labels = [], []

for i in range(len(train_df.Image)):
  train_image = cv2.imread(train_path + str(train_df.Image[i]))
  train_image = cv2.resize(train_image, (h, w))
  train_images.append(train_image)
  train_labels.append(train_df.Class[i])

test_images = []

for i in range(len(test_df.Image)):
  test_image = cv2.imread(test_path + str(test_df.Image[i]))
  test_image = cv2.resize(test_image, (h, w))
  test_images.append(test_image)

train_images = np.array(train_images)
test_images = np.array(test_images)

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_images, to_categorical(train_labels), test_size=0.2, random_state=42)

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

In [0]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False, 
    input_shape=(h, w, 3), 
    pooling='avg'
)
# base_model.summary()

In [11]:
base_model.trainable = False

model = Sequential([
  base_model,
  Dropout(0.2),
  Dense(4, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 8196      
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
batch_size = 32
epochs = 2

In [21]:
datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), validation_data= (X_test, y_test),
                    steps_per_epoch=len(X_train) / batch_size, epochs=epochs)

# training_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='training',seed=7)
# validation_generator = datagen.flow(X_train, y_train, batch_size=batch_size,subset='validation',seed=7)

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=200,
#         epochs= epochs,
#         validation_data= validation_generator,
#         validation_steps= 80)

Epoch 1/2
150/149 [==============================] - 743s 5s/step - loss: 0.6213 - acc: 0.7696 - val_loss: 0.7312 - val_acc: 0.7435
Epoch 2/2
150/149 [==============================] - 762s 5s/step - loss: 0.5861 - acc: 0.7796 - val_loss: 0.8021 - val_acc: 0.7218


In [0]:
# model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), callbacks=[EarlyStopping(min_delta=0.01)])

In [22]:
labels = model.predict(test_images)
print(labels[:4])

[[2.57640988e-01 2.78162986e-01 4.52965382e-04 4.63743001e-01]
 [8.95924211e-01 5.51426485e-02 1.06560256e-04 4.88265604e-02]
 [9.04207706e-01 7.27646891e-03 1.90586853e-03 8.66099894e-02]
 [9.98800278e-01 4.06225736e-05 1.82576659e-05 1.14088808e-03]]


In [23]:
label = [np.argmax(i) for i in labels]
print(label[:20])

[3, 0, 0, 0, 1, 1, 3, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 1, 3, 1]


In [24]:
class_label = [inverse_class_map[x] for x in label]
print(class_label[:3])

['misc', 'Food', 'Food']


In [25]:
submission = pd.DataFrame({ 'Image': test_df.Image, 'Class': class_label })
submission.head()

,Image,Class
0,image6245.jpg,misc
1,image10409.jpg,Food
2,image8692.jpg,Food
3,image10517.jpg,Food
4,image2580.jpg,Attire


In [0]:
submission.to_csv('sub.csv', index=False)